In [1]:
import glob
import os
from pathlib import Path

import polars as pl
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# Data preprocessing

**Steps**
1. Concat train/test data files.
2. Extract categorical features from data.
3. Scale numeric features and ordinal encode categorical features.

## Constants

In [7]:
DATA_PATH = Path("../data/")

## Step 1
**Concat train/test data files**

In [3]:
path_train = '../data/train'
path_test = '../data/test'
print(*[f'Список файлов в репозитории {x.split("/")[-1]}: {sorted(os.listdir(x))}' for x in [path_train, path_test]], sep='\n')
# Получим список путей к файлам в папке train
filenames_train = glob.glob(path_train + "/*.csv")
# Создадим список для записи считанных файлов train
data_files_train = []
# Считаем все файлы train и добавим их в список
for filename in filenames_train:
    data_files_train.append(pl.read_csv(filename))
# Объединим тренировочные данные в единый датасет
data_train = pl.concat(data_files_train)
# Выведем информацию о размерности полученных тренировочных данных
print('Размерность полных тренировочных данных составляет: {} строки и {} столбцов'.format(*data_train.shape))

# Получим список путей к файлам в папке train
filenames_test = glob.glob(path_test + "/*.csv")
# Создадим список для записи считанных файлов train
data_files_test = []
# Считаем все файлы train и добавим их в список
for filename in filenames_test:
    data_files_test.append(pl.read_csv(filename))
# Объединим тренировочные данные в единый датасет
data_test = pl.concat(data_files_test)

Список файлов в репозитории train: ['train_1.csv', 'train_10.csv', 'train_2.csv', 'train_3.csv', 'train_4.csv', 'train_5.csv', 'train_6.csv', 'train_7.csv', 'train_8.csv', 'train_9.csv']
Список файлов в репозитории test: ['test_1.csv', 'test_10.csv', 'test_2.csv', 'test_3.csv', 'test_4.csv', 'test_5.csv', 'test_6.csv', 'test_7.csv', 'test_8.csv', 'test_9.csv']
Размерность полных тренировочных данных составляет: 413194 строки и 189 столбцов


## Step 2 
**Extract categorical features from data.**

In [4]:
cat_unique_thresh = 150
cat_condition = data_train.select(pl.col("*").n_unique() < cat_unique_thresh).to_numpy()[0]
cat_columns = data_train[:, cat_condition].drop("target", "smpl", "id", strict=False).columns
num_columns = data_train.drop("target", "smpl", "id", *cat_columns).columns

# convert cat_columns to str 
# and num_columns columns to float32 to reduce memory consumption
data_train = data_train.with_columns("target", "smpl", "id", pl.col(cat_columns).cast(pl.Int16), pl.col(num_columns).cast(pl.Float32))
data_test = data_test.with_columns("smpl", "id", pl.col(cat_columns).cast(pl.Int16), pl.col(num_columns).cast(pl.Float32))
print(f"Number of categorical columns: {len(cat_columns)}")
print(f"Number of numeric columns: {len(num_columns)}")

Number of categorical columns: 12
Number of numeric columns: 174


## Step 3
**Scale numeric features and ordinal encode categorical features.**


Apply basic preprocessing, since data is already in good condition (no missing values, no outliers)

In [8]:
# Basic preprocessing
oe = OrdinalEncoder(dtype=np.int32, handle_unknown="use_encoded_value", unknown_value=-1)
ss = StandardScaler()

data_train[cat_columns] = oe.fit_transform(data_train[cat_columns])
data_train[num_columns] = ss.fit_transform(data_train[num_columns])

data_test[cat_columns] = oe.transform(data_test[cat_columns])
data_test[num_columns] = ss.transform(data_test[num_columns])

data_train.write_parquet(DATA_PATH / "train_preproc_2.parquet")
data_test.write_parquet(DATA_PATH / "test_preproc_2.parquet")